In [ ]:
"""
## Organize the "Thesis" ##
#  The approach is kind of similar to GMMs  # 
# Currently, don't know the perfect weights #

#Can you do some temporal crowd density (spatial) analysis on the data and give some observations

##Find the correct intersection point of the circle intersection with line function
## Make the function "plotSimulation" more smarter #plot the agent direction line as well -- i.e. the centre line
###Plot the max value point as well<<>> in the probability distribution map
##plot humans walk as form of circle with an arrow pointing towards velocity >> should be different for the agent

##Need to record data with a stable camera##<<
##Find walk- or person specific statistics

#Also check for the feasible point as well before assigning the new position e.g. agents goes inside obstacles
consider the human field of vision as well - we don't scan like lidars

Inertia , cognitive, 
Cromwell road Tesco
Time as a source of information for the goal sigma ..<<
############################
1. can we couple eye gaze - gives direction with semantic segmentation - gives scene understanding

####################  CALIBRATION BASED. ######################
1. A calibration process is required -- it very important
need a controller for velocity
width of average human (in pixel value)
average speed of humna (in walk and brisk walk) - 
->how much each step will be in terms of distance wise which varies say for gaussian distribution way
width of walkway passage and the shelf where items are kept
33. Define the limit condition so as to have real-life cases
1. there must be a higher distance limit for the obstacle scan

An agent class
Intrinsic parameter - behaviour mode, age, avgerage velocity
Extrinsic parameter - start position, end goal, obstacle map, velocity (mag+dir)

######################  FEATURE BASED. ########################
What are features to consider?
11. obstacle/people in front of if the obstacle is people then the velocity of approach 
-- currently not taking care of social interaction between 2 people
5. can we say goals of 2 types
    a. accurate
    b. directional (where people know in which direction to go don't know the exact path -> for new people or first timers)
"""

"""
Help Sites: Checking point inside a rectangle: https://stackoverflow.com/questions/2752725/finding-whether-a-point-lies-inside-a-rectangle-or-not
Removing duplicate point: https://stackoverflow.com/questions/35731289/function-to-remove-duplicates-from-a-list-python?lq=1
Writing text on matplotlib plot: https://matplotlib.org/api/_as_gen/matplotlib.pyplot.text.html
"""
%matplotlib inline
import math
import pickle
import threading
import numpy as np
import matplotlib.pyplot as plt

import pathPlanRRT as rrt
import agentSimulation as agentsim
import probabilityConv as probconv
import simulationEnv as envsim
import mathHelper as mh
import simPlot as simplot

from IPython.display import clear_output

def getStartPos(fromDoor = True):
    pos = [int(np.random.normal(650, 50, 1)), 1] # np.random.normal(centre_of_door, sigma = doorlen/6, 1), door_yaxis
    return pos

def remove_duplicates(x):
    z = []
    z.append(x[0])
    for i in range(1,len(x)):
        for y in range(0, i):
            if x[i][0] == x[y][0] and x[i][1] == x[y][1]:
                break
        else:
            z.append(x[i])
    return z  

In [ ]:
# Initialize the simulator and the environment
agent                  = agentsim.agent(start= np.array(getStartPos()), goal=np.array([500, 400]))
agent.simulation_env   = envsim.museumSimulationEnvironment()
human_data             = envsim.getActorTrajectory(plot = False)
human_path, agent_path = envsim.agentTimeSynchronize(human_data, agent_num = 1)

agent_path = list(reversed(agent_path))
# Initialize the simulating agent parameters
agent.avg_speed = 10                                         # need to convert average distance to pixel space # and get from agents statistics
agent.cur_pos   = agent.start = np.array(agent_path[5][2])   # where earlier pts was given by the planner
del_t           = 1                                          # step movement = del_t * avg_speed
del_vec         = np.array(agent_path[1][2]) - np.array(agent_path[0][2])
agent.cur_vel   = (del_vec/np.sqrt(sum(del_vec*del_vec.T)))*agent.avg_speed 
ref_path        = []
hpath           = [[] for i in range(len(human_path))]

for i in range(len(agent_path)-1):    # iterate over each point in the path
    #inter_dist    = agentsim.euclidDist(agent.cur_pos, pts[i+1])
    agent_num, cur_frame, cur_pos   = agent_path[i] 
    agent_num, goal_frame, goal_pos = agent_path[i+1]
    
    agent.goal     = np.array(goal_pos)
    old_vel        = agent.cur_vel
    new_vel        = agent.updateVelocity()
    agent.cur_pos  = agent.cur_pos + agent.cur_vel*del_t 
    
    agent.path.append(agent.cur_pos)
    ref_path.append(np.array(cur_pos))
    simplot.plotSimulation(agent = agent, obstacleList = agent.simulation_env, 
                           path1 = ref_path, path2 = None, human_path = hpath, 
                           agpath = human_path, cur_frame = cur_frame)
    
    print(" Position :: ", agent.cur_pos, 
          " velocity :: ", agent.cur_vel, 
          " angle :: ",    mh.vectorToAngle(agent.cur_vel))
    print(" Start ::",     agent.start, 
          " Goal :: ",     agent.goal)
    
    # get the other actor position    
    # update humans position and velocity in database 
    # calculate the crowd density as well

    # get all the parameters near the agent like static and dynamic obstacles (like object and humans), crowd density
    # based on the goal and conditions in the environment calculate the velocity 
    # calculate the next position based on the calculated velocity

    # get the ground truth path
    # plot both estimated and ground truth together

In [ ]:
#drawing ellipse in matplotlib: https://matplotlib.org/api/_as_gen/matplotlib.patches.Ellipse.html

from matplotlib.patches import Ellipse

sub = []
X = [ (1,2,1), (3,2,2), (3,2,4), (3,2,6) ]
fig = plt.figure(figsize=(18,9))
#ells = [Ellipse((1, 1), 4, 2, a) for a in angles]
for nrows, ncols, plot_number in X:
    f = fig.add_subplot(nrows, ncols, plot_number)
    sub.append(f)
    


    
ellipse = Ellipse((0,0), 50, 30, angle=0.0)
ellipse.set_alpha(0.5)
#e.set_clip_box(a.bbox)
ellipse.set_facecolor(np.random.rand(3))
sub[0].add_artist(ellipse)
sub[0].set_xlim((-40,40))    
sub[0].set_ylim((-40,40))
plt.show()

In [ ]:
"""
Simulation For Supermarket
"""
agent1  = agentsim.agent(start= np.array(getStartPos()), goal=np.array([500, 400]), cur_vel= np.array([5,-5]))
agent1.simulation_env = envsim.storeSimulationEnvironment()


#Planner part which generates a smooth path for the agent
planner = rrt.RRT(start=agent1.start, goal=agent1.goal, randArea=[0, 800], obsList=agent1.simulation_env['rect'], expandDis=10.0)
path    = planner.Planning(animation=False)
path_x  = [x for (x, y) in reversed(path)]
path_y  = [y for (x, y) in reversed(path)]
PathS   = rrt.PathSmoothing(path, 500, agent1.simulation_env['rect'])
rep_pts = np.array(PathS, dtype=np.int16)
pts     = np.array(remove_duplicates(rep_pts))
pts     = pts[::-1]  # reverse the order

agent1.avg_speed = 1 # need to convert average distance to pixel space
agent1.cur_pos   = agent1.start = pts[0]
del_t            = 1 # step movement = del_t * avg_speed
del_vec          = pts[1]-pts[0]
agent1.cur_vel   = (del_vec/np.sqrt(sum(del_vec*del_vec.T)))*agent1.avg_speed 

for i in range(len(pts)-1):
    inter_dist   = mh.euclidDist(agent1.cur_pos, pts[i+1])
    agent1.goal  = pts[i+1]
    while inter_dist > 1.0:
        old_vel        = agent1.cur_vel
        new_vel        = agent1.updateVelocity()
        agent1.cur_pos = agent1.cur_pos + agent1.cur_vel*del_t
        inter_dist     = mh.euclidDist(agent1.cur_pos, pts[i+1])
        agent1.path.append(agent1.cur_pos)
        simplot.plotSimulation(agent1, agent1.simulation_env, path, PathS)
        print("Position:: ", agent1.cur_pos, " :: velocity :: ", agent1.cur_vel, " :: angle :: ", mh.vectorToAngle(agent1.cur_vel))
        print("Start ::", agent1.start, "Goal:: ", agent1.goal)

#Save the scan data with obstacle distance and angle
#with open('scan_map', 'wb') as fp:
#    pickle.dump(scan_map, fp)

In [ ]:
xy = [(1, 278, (422.2490443488504, 492.0732197104761)),
      (1, 276, (420.9691656015115, 495.3031619374767)),]

yz = [(1, 278, (42.494, 42.04761)),
      (1, 280, (4.95, 49.4767)),]

tmp = [xy, yz,]
#print(tmp)
for l in tmp:
    for x,y,z in l:
        if y <277:
            print(l.remove((x,y,z)))
        #print(x,y,z)
    
#print(tmp)

In [ ]:
def getCrowdDensity():
    return 0

def updatePosition(self):
    #update the agent position
    return 0
    
"""
Funtion: Convert walkable path into probability sd- which can be got from semantic segmentation in real-life
Input: 

It will be probably be a unit function kind of
"""
def walkablePathToProbability():
    return 0


def activateAgent(agent1, obstacleList):
    rrt  = rrtplanner.RRT(start=agent1.start, goal=agent1.goal, randArea=[0, 800], obsList=obstacleList, expandDis=10.0)
    path = rrt.Planning(animation=False)
    smooth_path = rrtplanner.PathSmoothing(path, 500, obstacleList)
    
    path_x = [x for (x, y) in reversed(smooth_path)]
    path_y = [y for (x, y) in reversed(smooth_path)]
    i = 0
    while i<(len(path_x)-1):
        if agent1.step == False:
            #rrt.DrawGraph()
            #plt.plot(path_x[0:i+1], path_y[0:i+1], '-r')
            agent1.cur_pos = np.array([path_x[i], path_y[i]])
            agent1.cur_vel = np.array([path_x[i+1] - path_x[i], path_y[i+1] - path_y[i]])  
            agent1.obs_map = agent1.obstacleMap(obstacleList)       
            agent1.path.append(agent1.cur_pos)
            agent1.step = True
            i = i + 1


######### Depreciated Functions ############
def plotCircle(x, y, size):
    deg = list(range(0, 360, 5))
    deg.append(0)
    xl = [x + size * math.cos(math.radians(d)) for d in deg]
    yl = [y + size * math.sin(math.radians(d)) for d in deg]
    plt.plot(xl, yl, "-k")
    
    
def drawGraph(obsList, rnd=None):
    plt.clf()
    if rnd is not None:
        plt.plot(rnd[0], rnd[1], "^k")
    for node in nodeList:
        if node.parent is not None:
            plt.plot([node.x, nodeList[node.parent].x], [
                     node.y, nodeList[node.parent].y], "-g")

    plotObstacles(obsList)
    plt.plot(start.x, start.y, "xr")
    plt.plot(end.x, end.y, "xr")
    plt.axis([0, 800, 0, 800])
    plt.grid(True)
    plt.pause(0.01)
    
    
"""
Function: Returns distance of rectangle egdes (x1,y1) & (x2, y2) from a point (x,y)
Returns 0 if point is inside the rectangle
Help link: https://gamedev.stackexchange.com/questions/44483/how-do-i-calculate-distance-between-a-point-and-an-axis-aligned-rectangle
"""
def pointRectangleDistance(x, y, x1, y1, x2, y2):
    dx = 0.0
    dy = 0.0
    if (x < x1):
        dx = x1 - x
        if (y < y1):
            dy = y1 - y
            return math.sqrt(dx * dx + dy * dy)
        elif (y > y2):
            dy = y - y2
            return math.sqrt(dx * dx + dy * dy)
        else:
            return dx
        
    elif (x > x2):
        dx = x - x2
        if (y < y1):
            dy = y1 - y
            return math.sqrt(dx * dx + dy * dy)
        elif (y > y2):
            dy = y - y2
            return math.sqrt(dx * dx + dy * dy)
        else:
            return dx
        
    else:
        if (y < y1):
            return y1 - y
        elif (y > y2): 
            return y - y2
        else:
            return 0

def LineIntersectCircle(p,lsp,lep):
    # p is the circle parameter, lsp and lep is the two end of the line
    x0,y0,r0 = p
    x1,y1 = lsp
    x2,y2 = lep
    if x1 == x2:
        if abs(r0) >= abs(x1 - x0):
            p1 = x1, y0 - math.sqrt(r0**2 - (x1-x0)**2)
            p2 = x1, y0 + math.sqrt(r0**2 - (x1-x0)**2)
            inp = [p1,p2]
            # select the points lie on the line segment
            inp = [p for p in inp if p[1]>=min(y1,y2) and p[1]<=max(y1,y2)]
        else:
            inp = []
    else:
        k = (y1 - y2)/(x1 - x2)
        b0 = y1 - k*x1
        a = k**2 + 1
        b = 2*k*(b0 - y0) - 2*x0
        c = (b0 - y0)**2 + x0**2 - r0**2
        delta = b**2 - 4*a*c
        if delta >= 0:
            p1x = (-b - math.sqrt(delta))/(2*a)
            p2x = (-b + math.sqrt(delta))/(2*a)
            p1y = k*x1 + b0
            p2y = k*x2 + b0
            inp = [[p1x,p1y],[p2x,p2y]]
            # select the points lie on the line segment
            inp = [p for p in inp if p[0]>=min(x1,x2) and p[0]<=max(x1,x2)]
        else:
            inp = []
        return inp

    
def lineIntersectionCircle(circle, p1, p2):
    center,radius = circle
    dx, dy = p2[0] - p1[0], p2[1] - p1[1]

    a = dx**2 + dy**2
    b = 2 * (dx * (p1[0] - center[0]) + dy * (p1[1] - center[1]))
    c = (p1[0] - center[0])**2 + (p1[1] - center[1])**2 - radius**2

    discriminant = b**2 - 4 * a * c
    if (discriminant > 0):
        return None

    t1 = (-b + discriminant**0.5) / (2 * a)
    t2 = (-b - discriminant**0.5) / (2 * a)

    return [dx * t1 + p1[0], dy * t1 + p1[1]], [dx * t2 + p1[0], dy * t2 + p1[1]]

In [ ]:
import pygame
import numpy as np
 
pygame.init()
window = pygame.display.set_mode((800,800))
pygame.display.set_caption("Human Behaivour")
 
black = (0,0,0)
white = (255,255,255)
red   = (255, 25, 25)
blue  = (0, 255, 0)
green = (0, 0, 255)
 
clock = pygame.time.Clock()
 
def detectCollisions(x1,y1,w1,h1,x2,y2,w2,h2):
    if (x2+w2>=x1>=x2 and y2+h2>=y1>=y2):
        return True
    elif (x2+w2>=x1+w1>=x2 and y2+h2>=y1>=y2):
        return True
    elif (x2+w2>=x1>=x2 and y2+h2>=y1+h1>=y2):
        return True
    elif (x2+w2>=x1+w1>=x2 and y2+h2>=y1+h1>=y2):
        return True
    else:
        return False
    
class Sprite:
    def __init__(self,x,y,width,height):
        self.x=x
        self.y=y
        self.width=width
        self.height=height
 
    def render(self,collision):
        if (collision==True):
            pygame.draw.rect(window,red,(self.x,self.y,self.width,self.height))
        else:
            pygame.draw.rect(window,black,(self.x,self.y,self.width,self.height))

def drawCircle():
    pygame.draw.circle(window, red, (780, 780), 0)

Sprite1=Sprite(0, 0, 200, 50)
Sprite2=Sprite(300, 0, 200, 50)
Sprite3=Sprite(600, 0, 200, 50)
Sprite4=Sprite(0, 150, 50, 400)
Sprite5=Sprite(50+100, 150, 100, 400)
Sprite6=Sprite(150+200, 150, 100, 400)
Sprite7=Sprite(150+200+200, 150, 100, 400)
Sprite8=Sprite(150+200+200+200, 150, 50, 400)
Sprite9=Sprite(0, 50+100+400+100, 200, 250)
Sprite10=Sprite(300, 50+100+400+100, 200, 250)
 
moveX,moveY=0,0
gameLoop=True

startPosXY0 = [0, 0]
curPosXY0   = [0, 0]
targPosXY0  = [0, 0]
startPosXY1 = [0, 0]
curPosXY1   = [0, 0]
targPosXY1  = [0, 0]
####
#Info
#pygame module for drawing shapes
#pygame.draw.rect	—	draw a rectangle shape
#pygame.draw.polygon	—	draw a shape with any number of sides
#pygame.draw.circle	—	draw a circle around a point
#pygame.draw.ellipse	—	draw a round shape inside a rectangle
#pygame.draw.arc	—	draw a partial section of an ellipse
#pygame.draw.line	—	draw a straight line segment
#pygame.draw.lines	—	draw multiple contiguous line segments
#pygame.draw.aaline	—	draw fine antialiased lines
#pygame.draw.aalines	—	draw a connected sequence of antialiased lines
#######################################################################################################################
#Important Questions to be answered#
#1. How many actors do you need??
#2. What is path planning algo which you req.??
#3. what about dimension mapping to the real object to the one in the simulated one
#4. an outer circle which is proportional to the crowd density
#######################################################################################################################
#1. As its a graphical array, all the values should be in integers
#######################################################################################################################
#function for mapping point to free space or obstacle space
#function for person (circle) with the velocity vector (if possible proportional to speed)
#function for global path planning module (inputs are initialxy, finalxy and obstacleMap)
#function for local path planner which gives intermediate steps
#function for generating human like behaviour path considering nearby humans!!
#function which gives obstacle map i.e. obstacles with distance which are infront of the person
#function which gives random 2d points from a gaussian distribution given a mean, sigma
#HIGH level function which controls motion (i.e. path planning and behaivour) of each of the actors
#function which traces the path taken by each actor

def getStartPos(fromDoor = True):
    pos = [int(np.random.normal(650, 50, 1)), 800] # np.random.normal(centre_of_door, sigma = doorlen/6, 1), door_yaxis
    return pos

def getIntermedPos(curPos, targetPos, step):
    x = np.linspace(curPos[0], targetPos[0], 200)
    y = np.linspace(curPos[1], targetPos[1], 200)
    intPos = [int(x[step]),int(y[step])]
    return intPos

targPosXY0 = [500,400]
curPosXY0  = startPosXY0 = getStartPos()

targPosXY1 = [700, 400]
curPosXY1  = startPosXY1 = getStartPos()
steps = 0

while gameLoop: 
    for event in pygame.event.get():
        if (event.type==pygame.QUIT):
            gameLoop=False
        if (event.type==pygame.KEYDOWN):
            if (event.key==pygame.K_LEFT):
                moveX = -4
            if (event.key==pygame.K_RIGHT):
                moveX = 4
            if (event.key==pygame.K_UP):
                moveY = -4
            if (event.key==pygame.K_DOWN):
                moveY = 4
 
        if (event.type==pygame.KEYUP):
            if (event.key==pygame.K_LEFT):
                moveX=0
            if (event.key==pygame.K_RIGHT):
                moveX=0
            if (event.key==pygame.K_UP):
                moveY=0
            if (event.key==pygame.K_DOWN):
                moveY=0
                
    window.fill(white)
    Sprite1.x+=moveX
    Sprite1.y+=moveY
    collisions=detectCollisions(Sprite1.x,Sprite1.y,Sprite1.width,Sprite1.height,Sprite2.x,Sprite2.y,Sprite2.width,Sprite2.height)
    Sprite1.render(collisions)
    Sprite2.render(False)
    Sprite3.render(False)
    Sprite4.render(False)
    Sprite5.render(False)
    Sprite6.render(False)
    Sprite7.render(False)
    Sprite8.render(False)
    Sprite9.render(False)
    Sprite10.render(False)
    pygame.draw.circle(window, red, curPosXY0, 5, 0)
    pygame.draw.circle(window, blue, curPosXY1, 5, 0)
    
    steps = steps + 1
    curPosXY0 = getIntermedPos(startPosXY0, targPosXY0, steps)
    curPosXY1 = getIntermedPos(startPosXY1, targPosXY1, steps)
    
    pygame.display.flip()
    clock.tick(50)
    
    if steps == 199: break
    
pygame.quit()